In [ ]:
!pip install pandas power-grid-model-ds[visualizer] --quiet

# ⚡ Advanced Power Grid Workshop: Solving an Overload Scenario

You're a senior grid analyst at GridNova Utilities, overseeing a legacy radial distribution network in a semi-rural region. Recently, customer load growth has increased dramatically, particularly in areas served by a single long feeder. This has pushed some branches past their capacity, triggering repeated overloads.

Your task: augment the grid by adding a second substation and relieving the overloaded feeder through targeted switching.

This hands-on simulation walks you through each step of diagnosing, planning, and solving this overload using the Power Grid Model DS library.

## 🎯 Workshop Goals
- Detect a line overload after load increase.
- Find a suitable node to connect a new substation.
- Trace the overloaded route.
- Strategically open a line to reroute power and relieve the feeder.



In [ ]:
import numpy as np

from dataclasses import dataclass
from power_grid_model_ds import Grid, GraphContainer
from power_grid_model_ds.arrays import LineArray, NodeArray, SourceArray
from power_grid_model_ds.enums import NodeType
from power_grid_model_ds.visualizer import visualize

# 🧪 Step 1: Extend the Data Model
Goal: Add coordinate fields and tracking for simulated voltages and line currents.

You’ll subclass NodeArray and LineArray to add:

- x, y coordinates for spatial logic and plotting
- u for node voltage results
- i_from for line currents
- A computed .is_overloaded property for easy filtering

In [ ]:
# 📦 Extend the grid with x, y, u (node) and i_from (line)
from numpy.typing import NDArray

class MyNodeArray(NodeArray):
    _defaults = {"x": 0.0, "y": 0.0, "u": 0.0}
    x: NDArray[np.float64]
    y: NDArray[np.float64]
    u: NDArray[np.float64]

class MyLineArray(LineArray):
    _defaults = {"i_from": 0.0, "overload_status": 0}
    i_from: NDArray[np.float64]
    overload_status: NDArray[np.int8]

    def set_overload_status(self):
        """Set the overload status based on the current and nominal current."""
        self.overload_status = np.where(self.i_from > self.i_n, 1, 0)

    @property
    def is_overloaded(self) -> NDArray[np.bool_]:
        """Check if the line is overloaded."""
        self.set_overload_status()
        return self.overload_status == 1

@dataclass
class MyGrid(Grid):
    node: MyNodeArray
    line: MyLineArray
    graphs: GraphContainer

# 🏗️ Step 2: Load and Prepare the Grid
Goal: Load a synthetic medium-voltage grid from the provided data.

You'll use pandas to load the data from included CSV files and then:
- Create an empty Grid object using your custom MyGrid class
- Create and fill NodeArray, LineArray and LoadArray objects with data from the CSV files and append them to your Grid
- Using a SourceArray, add a voltage source to each substation in the Grid
- You’ll then compute a power flow on the synthetic grid to get realistic line currents

In [ ]:
from helper import load_dummy_grid

grid = load_dummy_grid(MyGrid)

![input_network.png](input_network.png)

# 🧯 Step 3: Detect the Overload
Goal: Identify which line(s) are exceeding their rated current.

You’ll:

- Use the .is_overloaded property to find the problem
- Isolate the feeder route to the affected line

This gives a clear target for where you need to intervene.

In [ ]:
# Check the grid for capacity issues
# 1. Use the PowerGridModelInterface to calculate power flow
# 2. Update the grid with the calculated values
# 3. Return the lines (LineArray) that are overloaded

# Hint: You can use the `is_overloaded` property of the `MyLineArray` class to check for overloaded lines.
# Hint: https://power-grid-model-ds.readthedocs.io/en/stable/quick_start.html#performing-power-flow-calculations

def check_for_capacity_issues(grid: Grid) -> LineArray:
    """Check for capacity issues on the grid.
    Return the lines that with capacity issues.
    """

print(check_for_capacity_issues(grid))

# %load solutions/advanced_3_check_for_capacity_issues.py

In [ ]:
visualize(grid)

# 🧭 Step 4: Plan a Relief Strategy
![input_network_with_overload.png](input_network_with_overload.png)

Goal: Place a second substation near the overloaded path.

You’ll:
- Add a new substation at a specified location
- Connect it to the closest node in the network
- Visually inspect the grid with visualize()

This substation will act as a new injection point for rerouting load.


In [ ]:
def build_new_substation(grid: Grid, location: tuple[float, float]) -> NodeArray:
    """Build a new substation at the given location.
    Return the new substation.
    """

# %load solutions/advanced_4_build_new_substation.py

# 🔗 Step 5: Trace and Analyze the Overloaded Route
Goal: Identify a switchable line along the route from the new node to the old substation.

You’ll:
- Use the graph interface to trace the shortest path from the new substation to the original
- Calculate cumulative load along the path
- Find a cut point where the remaining load toward the old substation is < 2 MW

This ensures you relieve the feeder while maintaining supply continuity.

In [ ]:
def get_all_congested_routes(grid: Grid) -> list[NodeArray]:
    """Get all routes that originate from a given substation node."""

# %load solutions/advanced_5_1_get_all_congested_routes.py

In [ ]:
def find_connection_point(route: NodeArray, new_substation: NodeArray) -> NodeArray:
    """Calculate the connection point for the new route.
    This should be the geographically closest node to the new substation.
    """

# %load solutions/advanced_5_2_find_connection_point.py

In [ ]:
def connect_to_route(grid: Grid, connection_point: NodeArray, new_substation: NodeArray) -> None:
    """Connect the new substation node to the connection point.
    """

# %load solutions/advanced_5_3_connect_to_route.py

# ✂️ Step 6: Open the Right Line
Goal: Deactivate a line to offload the original feeder.

You’ll:
- Identify and deactivate the line between two nodes on the critical path
- Rerun power flow after the switch
- Confirm the overload is resolved

This final step demonstrates how network topology and load can be managed dynamically with switching and distributed generation.



In [ ]:
def optimize_route_transfer(grid: Grid, connection_point: NodeArray, new_substation: NodeArray) -> None:
    """Attempt to optimize the route transfer moving the naturally open point (NOP) upstream towards the old substation.
    This way, the new substation will take over more nodes of the original route.
    """
    # Get the path from the connection point to the old substation
    ...

    # filter the first branch in the path
    ...

    # Iterate over the path and check if the route is still overloaded
    for from_node, to_node in zip(path[0:-1], path[1:]):
        # Check if the route is still overloaded
        ...
        
        # Move the Open Point (NOP) upstream
        ...
    
    grid.set_feeder_ids()

# %load solutions/advanced_6_optimize_route_transfer.py

In [ ]:
def transfer_routes(grid: Grid, new_substation: NodeArray) -> NodeArray:
    """Migrate a subset of the routes of the old substation to the new substation.
    Each route can be migrated fully or partially.

    """
    congested_routes = get_all_congested_routes(grid)

    for route in congested_routes:
        closest_node = find_connection_point(
            route=route,
            new_substation=new_substation
        )

        connect_to_route(
            grid=grid,
            connection_point=closest_node,
            new_substation=new_substation,
        )

        optimize_route_transfer(
            grid=grid,
            connection_point=closest_node,
            new_substation=new_substation)
        
        print(f"Connected new substation to node {closest_node.id}")

transfer_routes(grid=grid, new_substation=new_substation)

In [ ]:
print(check_for_capacity_issues(grid))

In [ ]:
visualize(grid)

# ✅ Wrap-Up
You’ve just:

Diagnosed a power system constraint
- Planned and executed a grid topology change
- Verified success with power flow simulations

We hope you enjoyed working with Power Grid Model DS and would love to hear your feedback